## Queries

In [15]:
import pandas as pd

queries = pd.read_csv("../Data/SIGIR_queries_IT.csv")
queries.head()

,QID,Query,Prompt Type,Task Sentiment
0,qGEN1,Chi era il padre di Micerino ?,General,NaN
1,qGEN2,Il figlio di Chefren,General,NaN
2,qGEN3,Il padre di Micerino,General,NaN
3,qGEN4,Micerino,General,NaN
4,qGEN5,Quanto e alta la tomba di Cheope?,General,NaN


## Bing

In [3]:
import json
from tqdm.notebook import tqdm
import httpx
from datetime import datetime

f = open("API_keys.json")
data = json.load(f)

key1 = data["bing"]["key1"]
SERP_endpoint = data["bing"]["SERP_endpoint"]
location = data["bing"]["location"]

f.close()

headers = {
            'Ocp-Apim-Subscription-Key': key1,
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'Accept-Encoding': 'gzip, deflate',
            'Accept-Language': 'en-US,en;q=0.9',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
        }

SERP_results = []
today = datetime.now()
today = today.strftime("%Y_%m_%d")

for _, row in tqdm(queries.iterrows(), total=len(queries)):
    qid = row["QID"]
    query = row["Query"]
    prompt_type = row["Prompt Type"]
    params = {
        'q': query,
        'count': 10, # number of results to be displayed
        'setLang': 'it-IT',
        'mkt':'it-IT'
    }

    SERP_response = httpx.get(url=SERP_endpoint, headers=headers, params=params)
    try:
        SERP_result_set = SERP_response.json()
        rank = 1
        asked_query = SERP_result_set['queryContext']['originalQuery']
        for result in SERP_result_set['webPages']['value']:
            web_title =  result["name"]
            web_snippet = result["snippet"]
            SERP_results.append([qid, asked_query, prompt_type, "Bing", web_title + ". " + web_snippet, rank, today])
            rank += 1
    except Exception as error:
        print(error)
        SERP_results.append([qid, asked_query, prompt_type, "Bing", None, None, today])

SERP_df = pd.DataFrame(SERP_results, columns=["QID", "Query", "Prompt Type", "IAS", "Resp", "Rank", "date_generated"])
SERP_df.to_csv("../Data/SIGIR_bing_resp.csv", index=False)

  0%|          | 0/176 [00:00<?, ?it/s]

## ChatGPT

In [16]:
from openai import OpenAI
import json
from tqdm.notebook import tqdm
from datetime import date

f = open("API_keys.json")
data = json.load(f)

API_KEY = data["chatgpt"]["api_key3"]
f.close()

client = OpenAI(
    # This is the default and can be omitted
    api_key=API_KEY,
)

# query = "Hello"

# model_id = 'gpt-4o-mini'
model_id = 'chatgpt-4o-latest'

def get_gpt_resp(query):
    response = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "Follow these two instructions in all your responses: 1. Use Italian language only; 2. Do not use English except in programming language if any."
            },
            {
                "role": "user",
                "content": query,
            }
        ],
            model=model_id,
        )

    gpt_resp = response.choices[0].message.content

    return gpt_resp

# get_gpt_resp("Hello how are you?")

In [17]:
LLM_resp = []

for _, row in tqdm(queries.iterrows(), total=len(queries)):
    qid = row["QID"]
    query = row["Query"]
    prompt_type = row["Prompt Type"]

    result = get_gpt_resp(query)
    LLM_resp.append([qid, query, prompt_type, "GPT", result, date.today()])

LLM_resp_df = pd.DataFrame(LLM_resp, columns=["QID", "Query", "Prompt Type", "IAS", "Resp", "date_generated"])
LLM_resp_df.to_csv("../Data/SIGIR_gpt_resp.csv", index=False)

LLM_resp_df

  0%|          | 0/176 [00:00<?, ?it/s]

,QID,Query,Prompt Type,IAS,Resp,date_generated
0,qGEN1,Chi era il padre di Micerino ?,General,GPT,"Il padre di Micerino era Chefren, noto anche c...",2025-02-05
1,qGEN2,Il figlio di Chefren,General,GPT,Il figlio di Chefren era **Micerino** (Menkaur...,2025-02-05
2,qGEN3,Il padre di Micerino,General,GPT,"Il padre di Micerino era Chefren, noto anche c...",2025-02-05
3,qGEN4,Micerino,General,GPT,"Micerino, noto anche come Menkaura, fu un fara...",2025-02-05
4,qGEN5,Quanto e alta la tomba di Cheope?,General,GPT,"La tomba di Cheope, ovvero la Grande Piramide ...",2025-02-05
...,...,...,...,...,...,...
171,qEM62,Di cosa si occupa il WWF?,Emotionally Charged,GPT,Il WWF (World Wide Fund for Nature) è un'organ...,2025-02-05
172,qEM63,Cosa Ã¨ Ocean Cleanup ?,Emotionally Charged,GPT,**The Ocean Cleanup** è un'organizzazione no-p...,2025-02-05
173,qEM64,Quali sono le regole per tenere le spiagge pul...,Emotionally Charged,GPT,Mantenere le spiagge pulite è fondamentale per...,2025-02-05
174,qEM65,Di cosa si occupa la WWF,Emotionally Charged,GPT,Il WWF (World Wide Fund for Nature) è un'organ...,2025-02-05


## Google Gemma

In [1]:
from huggingface_hub import login

login()

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Specify the LLM model we'll be using
model_name = "google/gemma-2b-it"

# Configure for GPU usage
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)

# Load the tokenizer for the chosen model
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Create a pipeline object for easy text generation with the LLM
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

def get_gemma_resp(query):
  """Sends a conversation history to the AI assistant and returns the answer.

  Args:
    messages (list): A list of dictionaries, each with "role" and "content" keys.

  Returns:
    str: The answer from the AI assistant.
  """

  messages = [
      {"role":"user", "content":""},
      {"role": "assistant", "content": "Follow these two instructions in all your responses: 1. Use Italian language only; 2. Do not use English except in programming language if any."},
      {"role": "user", "content": query}
  ]

  generation_args = {
      # "max_new_tokens": 256,     # Maximum length of the response
      "return_full_text": False,      # Only return the generated text
  }

  output = pipe(messages, **generation_args)
  return output[0]['generated_text']

# gen_resp("Hi!")

In [15]:
from tqdm.notebook import tqdm
from datetime import date
import pandas as pd

def create_resp_file():
  LLM_resp = []

  for _, row in tqdm(queries.iterrows(), total=len(queries)):
      qid = row["QID"]
      query = row["Query"]
      prompt_type = row["Prompt Type"]

      result = get_gemma_resp(query)
      LLM_resp.append([qid, query, prompt_type, "Gemma", result, date.today()])

  LLM_resp_df = pd.DataFrame(LLM_resp, columns=["QID", "Query", "Prompt Type", "IAS", "Resp", "date_generated"])

  return LLM_resp_df

In [ ]:
df = create_resp_file()
file_name = "../Data/SIGIR_gemma_resp.csv"
df.to_csv(file_name, index=False)